# Lab 2: Detecting Bad Sensors in Power System Monitoring

In this lab, our goal is to detect bad sensor data measured on the IEEE 14 bus test
system shown below. The power flow equations that couple the voltages and power flows are 
nonlinear in nature, as discussed in class. We will load the sensor data from the
file 'sensorData14Bus.csv', and utilize SVM to perform the bad data detection.
We aim to understand how various parameters such as the nature of the corrupt data,
the number of corrupt data, etc., affect our abilities to classify the data.

<img src="IEEE14bus.png">

### First, we need to call the needed libraries

In [15]:
import numpy as np
import os
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from IPython.display import Image, display
import pandas as pd


### Loading the data 

Load the sensor data from the IEEE 14 bus test system, that has 14 buses
 and 20 branches. The data has been generated by adding a small noise
 to feasible voltages and power flows.
     
     Columns 1-14 contain bus voltage magnitudes.
     
     Columns 15-28 contain bus voltage phase angles.
     
     Columns 29-48 contain real power flow on all branches.
     
     Columns 49-68 contain reactive power flow on all branches.

In [16]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 3, 5, 10, 11, 15, 17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26


### Curroption Models 

Intentionally corrupt the first 'nCorrupt' rows of the data by adding
 a quantity to one or two sensor measurements that is not representative of
 our error model. We aim to study what nature of corruption is easier
 or difficult to detect.
 Specifically, we shall study 3 different models:
 
     1. 'corruptionModel' = 1 : Add a random number with a bias to one of the measurements.
     
     2. 'corruptionModel' = 2 : Add a random number without bias to one of the measurements.
     
     3. 'corruptionModel' = 3 : Add a random number with a bias to both the measurements.
     
In all these cases, we will multiply the sensor data by either a uniform or a normal random number multiplied by 'multiplicationFactor'.


In [17]:
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.
voltageMagnitudeColumn = lambda ii: ii

voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 2

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())


 It is always a good practice to scale your data to run SVM. Notice that we are
 cheating a little when we scale the entire data set 'X', because our training and
 test sets are derived from 'X'. Ideally, one would have to scale the training
 and test sets separately. Create the appropriate labels and shuffle the lists 'X' and 'Y' together.


In [18]:

X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)


Recall from the first lab that 'test_size' determines what fraction of the data becomes your test set.

## Task 1 (10 points)

Split the dataset into two parts: training and testing.
Store the training set in the variables 'trainX' and 'trainY'.
 Store the testing set in the variables 'testX' and 'testY.
 Reserve 20% of the data for testing.
The function 'train_test_split' may prove useful.


In [19]:
# Enter your code here
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)

## Task 2 (10 points)

 Define the support vector machine classifier and train on the variables 'trainX' and 'trainY'. Use the SVC library from sklearn.svm. Only specify three hyper-parameters: 'kernel', 'degree', and 'max_iter'. Limit the maximum number of iterations to 100000 at the most. Set the kernel to be a linear classifier first. You may have to change it to report the results with other kernels. The parameter 'degree' specifies the degree for polynomial kernels. This parameter is not used for other kernels. The functions 'svm.SVC' and 'fit' will prove useful.



In [20]:
# Enter your code here
classifier = svm.SVC(kernel='linear', degree=1, max_iter=100000)
classifier.fit(trainX, trainY)


SVC(degree=1, kernel='linear', max_iter=100000)

## Task 3 (10 points)

Predict the labels on the 'testX' dataset and store them in 'predictY'.


In [21]:
# Enter your code here
predictY = classifier.predict(testX)

## Task 4 (10 points)

Print the 'classification_report' to see how well 'predictY' matches with 'testY'.


In [22]:
# Enter your code here
print(classification_report(testY, predictY))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       671
         1.0       1.00      0.91      0.96       329

    accuracy                           0.97      1000
   macro avg       0.98      0.96      0.97      1000
weighted avg       0.97      0.97      0.97      1000



Print svm's internal accuracy score as a percentage.

In [23]:
# Enter your code here
print('classifier.score: ',classifier.score(testX, testY))

classifier.score:  0.972


## Task 5

We would like to compare 'classification_report' with this score for various runs. Let us consider the following cases: 

### Case 1:

Only have sensor measurements from the first 5 branches. Choose option 1 in the 'columnsToCorruptOption'. Examine how well linear kernels perform when 'corruptionModel' = 1, 'corruptionModel' = 2, and 'corruptionModel'= 3. In case linear kernels do not perform well, you may try 'rbf' or polynomial kernels with degree 2.

### Case 2:

Choose 'corruptionModel = 1' with 'linear' kernel. Does it pay to monitor voltage magnitudes than power flows? In other words, do you consistently get better results when you choose 'columnsToCorruptOption' as 2? Make these judgements using the average score of at least 5 runs.


#### Your task is to investigate the above two cases. You may add a few 'Markdown' and 'Code' cells below with your comments, code, and results. You can also report your results as a pandas DataFrame. You are free to report your results in your own way.

In [51]:
def load_data(nBuses=14, nBranches=20):
    # Select the bus numbers you monitor. For convenience, we have selected it for you.
    # The '-1' makes them columns as per Python's convention of starting to number
    # from 0.
    busesToSample = np.array([1, 2, 5, 10, 13]) - 1
    columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

    # Select the branches that you monitor.
    branchesToSample = np.array([1, 3, 5, 10, 11, 15, 17, 20]) - 1
    columnsForBranches = np.concatenate((branchesToSample + 28,
                                        branchesToSample + 48))

    # Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
    # specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
    # separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
    # with each column typecast as 'np.float32'.
    X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                      usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                      max_rows=5000)

    nDataPoints = np.shape(X)[0]
    nFeatures = np.shape(X)[1]

    print("Loaded sensor data on IEEE 14 bus system.")
    print("Number of data points = %d, number of features = %d"
          % (nDataPoints, nFeatures))

    return X


def add_corruption(X, corruptionModel=1, columnsToCorruptOption=2, multiplicationFactor=0.5):
    # Choose a corruption model.
    nDataPoints = np.shape(X)[0]
    nFeatures = np.shape(X)[1]
    nCorrupt = int(nDataPoints/3)

    # Choose which data to tamper with, that can be a voltage magnitude,
    # voltage phase angle, real power flow on a branch, reactive power flow
    # on a branch. We create functions to extract the relevant column to
    # corrupt the corresponding data in the 'ii'-th bus or branch.
    def voltageMagnitudeColumn(ii): return ii

    def voltageAngleColumn(ii): return ii + np.shape(busesToSample)[0]

    def realPowerColumn(ii): return ii + 2*np.shape(busesToSample)[0]
    def reactivePowerColumn(ii): return ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

    # Encode two different kinds of columns to corrupt.
    # Option 1: Corrupt real power columns only.
    # Option 2: Corrupt real power and voltage magnitude.

    if columnsToCorruptOption == 1:
        columnsToCorrupt = [realPowerColumn(1),
                            realPowerColumn(2)]
    else:
        columnsToCorrupt = [voltageMagnitudeColumn(0),
                            realPowerColumn(1)]

    # Corrupt the data appropriately, given the options.
    for index in range(nCorrupt):

        if corruptionModel == 1:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.rand())
        elif corruptionModel == 2:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.randn())
        else:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.rand())
            X[index, columnsToCorrupt[1]] \
                *= (1 + multiplicationFactor * np.random.rand())
    return X


def preprocess_data(X):
    nDataPoints = np.shape(X)[0]
    nFeatures = np.shape(X)[1]
    nCorrupt = int(nDataPoints/3)

    X = preprocessing.StandardScaler().fit_transform(X)

    # Create the labels as a column of 1's for the first 'nCorrupt' rows, and
    # 0's for the rest.
    Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))

    # Shuffle the features and the labels together.
    XY = list(zip(X, Y))
    np.random.shuffle(XY)
    X, Y = zip(*XY)

    return X, Y


def train_and_test_svm(X, Y, kernel='linear', degree=1, printResult=True):
    trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)

    classifier = svm.SVC(kernel=kernel, degree=degree, max_iter=100000)

    classifier.fit(trainX, trainY)

    predictY = classifier.predict(testX)

    report = classification_report(testY, predictY)
    score = classifier.score(testX, testY)

    if(printResult):
        print(report)
        print('classifier.score: ', score)

    return report, score


## Case1

| kernel | corruptionModel | accuracy |
| ------ | --------------- | -------- |
| linear | 1.0             | 0.979    |
| linear | 2.0             | 0.658    |
| linear | 3.0             | 0.970    |
| rbf    | 1.0             | 0.941    |	
| rbf    | 2.0             | 0.831    |	
| rbf    | 3.0             | 0.966    |	
| poly   | 1.0             | 0.820    |
| poly   | 2.0             | 0.850    |
| poly   | 3.0             | 0.825    |

- Data are in dataFrame.
- For corruptionModel = 1 and 3, linear has the best accracy.
- For corruptionModel = 2, poly with degree = 2 has the best accracy.
- Rbf has the best average accuracy for all corruptionModel.

In [55]:
dataFrame = pd.DataFrame(
    {
        "kernel": [],
        "corruptionModel": [],
        "accuracy": [],
        "report": [],
    }
)


corruptionModels = [1, 2, 3]
kernels = [('linear', 0), ('rbf', 0), ('poly', 2)]


for kernel in kernels:
    for corruptionModel in corruptionModels:
        print(f"------------{kernel[0]} corruptionModel={corruptionModel}------------")

        X = load_data(nBranches=5)
        X = add_corruption(X, corruptionModel=corruptionModel)
        X, Y = preprocess_data(X)
        report, score = train_and_test_svm(X, Y, kernel=kernel[0], degree=kernel[1])

        dataFrame = dataFrame.append(
            {
                "kernel": kernel[0],
                "corruptionModel": corruptionModel,
                "accuracy": score,
                "report": report
            }, ignore_index=True
        )

        print("------------------------------------------------")


display(dataFrame)


------------linear corruptionModel=1------------
Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       683
         1.0       1.00      0.93      0.97       317

    accuracy                           0.98      1000
   macro avg       0.99      0.97      0.98      1000
weighted avg       0.98      0.98      0.98      1000

classifier.score:  0.979
------------------------------------------------
------------linear corruptionModel=2------------
Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26


/home/lspss95207/.local/share/virtualenvs/Part2-wSLtOOJs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lspss95207/.local/share/virtualenvs/Part2-wSLtOOJs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lspss95207/.local/share/virtualenvs/Part2-wSLtOOJs/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

              precision    recall  f1-score   support

         0.0       0.66      1.00      0.79       658
         1.0       0.00      0.00      0.00       342

    accuracy                           0.66      1000
   macro avg       0.33      0.50      0.40      1000
weighted avg       0.43      0.66      0.52      1000

classifier.score:  0.658
------------------------------------------------
------------linear corruptionModel=3------------
Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       651
         1.0       1.00      0.92      0.96       349

    accuracy                           0.97      1000
   macro avg       0.98      0.96      0.97      1000
weighted avg       0.97      0.97      0.97      1000

classifier.score:  0.97
------------------------------------------------
------------rbf corruptionModel=1------------
Loaded s

/home/lspss95207/.local/share/virtualenvs/Part2-wSLtOOJs/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90       669
         1.0       1.00      0.55      0.71       331

    accuracy                           0.85      1000
   macro avg       0.91      0.77      0.80      1000
weighted avg       0.88      0.85      0.84      1000

classifier.score:  0.85
------------------------------------------------
------------poly corruptionModel=3------------
Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89       693
         1.0       1.00      0.43      0.60       307

    accuracy                           0.82      1000
   macro avg       0.90      0.71      0.74      1000
weighted avg       0.86      0.82      0.80      1000

classifier.score:  0.825
------------------------------------------------


,kernel,corruptionModel,accuracy,report
0,linear,1.0,0.979,precision recall f1-score ...
1,linear,2.0,0.658,precision recall f1-score ...
2,linear,3.0,0.970,precision recall f1-score ...
3,rbf,1.0,0.941,precision recall f1-score ...
4,rbf,2.0,0.831,precision recall f1-score ...
5,rbf,3.0,0.966,precision recall f1-score ...
6,poly,1.0,0.820,precision recall f1-score ...
7,poly,2.0,0.850,precision recall f1-score ...
8,poly,3.0,0.825,precision recall f1-score ...
